# Natasha
[Ссылка на репозиторий](https://github.com/natasha/natasha) 
Natasha решает основные NLP задачи для русского языка: токенизация, сегментация по предложениям, word embedding, классификация частей речи, лемматизация, нормализация фраз, парсинг синтаксиса, нахождение именованных сущностей, извлечение фактов. Качество по каждой задаче аналогично или лучше, чему текущие результаты SOTA для русского языка. Наташа - не исследовательский проект, основные технологии созданы для производства. Мы уделили внимание размеру модели, использованию оперативной памяти и производительности. Модель запускается на процессоре, используя интерфейс Numpy. 

Наташа объединяет библиотеки проекта под одним удобным API:
- **Razdel** - токен или сегментированное предложение.
- **Navec** - компактный эмбеддинг русских слов.
- **Slovnet** - современный инструмент на основе глубоко обучения для задач русскоязычного NLP, а также компактная модель для морфологии, синаксиса и нахождения именованных сущностей.
- **Yargy** - аналог парсера Tomita для извлечения фактов.
- **Ipymarkup** - NLP визуализатор для именованных сущностей и синтаксической маркировки.

## Install
Natasha supports Python 3.5+ and PyPy3:
```bash
$ pip install natasha
```

## Usage
Импорт, инициализация модулей, построение `Doc` объекта.

In [12]:
from natasha import (
    Segmenter,
    MorphVocab,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    PER,
    NamesExtractor,
    Doc
)

In [13]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)

text = 'Меркатто-Веккьо - торговый центр Флоренции. Антонио Пуччи, поэт, живший в XIV-м веке, так описал Меркато-Веккьо: "Врачи, готовые исцелить любую хворь, торговцы тканями, продавцы свиных колбас и аптекари". Во времена Римской Империи на месте Меркато был форум, и только в документах 1030 года он впервые назван рынком. Самым большим спросом здесь пользовались пшеница и шерсть. Однако если покупателям надоедало бродить по лавкам, они могли и развлечься. Иль-Панормита, поэт XV века, написал об этом так: "Там, в глубине, стоит веселый бордель, который ты узнаешь по запаху".'
doc = Doc(text)

## Segmentation
Разделение текста на токены и предложения. Определяет токены и предложения для `doc`. Использует `Razdel` внутри.

In [14]:
doc.segment(segmenter)
print(doc.tokens[:5])

[DocToken(stop=15, text='Меркатто-Веккьо'), DocToken(start=16, stop=17, text='-'), DocToken(start=18, stop=26, text='торговый'), DocToken(start=27, stop=32, text='центр'), DocToken(start=33, stop=42, text='Флоренции')]


In [15]:
print(doc.sents[:5])

[DocSent(stop=43, text='Меркатто-Веккьо - торговый центр Флоренции.', tokens=[...]), DocSent(start=44, stop=204, text='Антонио Пуччи, поэт, живший в XIV-м веке, так опи..., tokens=[...]), DocSent(start=205, stop=317, text='Во времена Римской Империи на месте Меркато был ф..., tokens=[...]), DocSent(start=318, stop=376, text='Самым большим спросом здесь пользовались пшеница ..., tokens=[...]), DocSent(start=377, stop=453, text='Однако если покупателям надоедало бродить по лавк..., tokens=[...])]


## Morphology
Для каждого токена извлекается богатая обозначеная морфология. Зависит от глубины сегментации. Определяет `pos` и `feats` свойства для `doc.tokens`. Использует **Slovnet morphology model** внутри.

Используйте `morph.print()` чтобы визуализировать морфологическую разметку.

In [16]:
doc.tag_morph(morph_tagger)
print(doc.tokens[:5])

[DocToken(stop=15, text='Меркатто-Веккьо', pos='PROPN', feats=<Inan,Nom,Masc,Sing>), DocToken(start=16, stop=17, text='-', pos='PUNCT'), DocToken(start=18, stop=26, text='торговый', pos='ADJ', feats=<Nom,Pos,Masc,Sing>), DocToken(start=27, stop=32, text='центр', pos='NOUN', feats=<Inan,Nom,Masc,Sing>), DocToken(start=33, stop=42, text='Флоренции', pos='PROPN', feats=<Inan,Gen,Fem,Sing>)]


In [17]:
doc.sents[0].morph.print()

     Меркатто-Веккьо PROPN|Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
                   - PUNCT
            торговый ADJ|Case=Nom|Degree=Pos|Gender=Masc|Number=Sing
               центр NOUN|Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
           Флоренции PROPN|Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
                   . PUNCT


## Lemmatization
Лемматизирует каждый токен. Зависит от морфологической глубины. Определяет `lemma` свойства для `doc.tokens`. Использует **Pymorphy** внутри.

In [18]:
for token in doc.tokens:
    token.lemmatize(morph_vocab)

print(doc.tokens[:5])

[DocToken(stop=15, text='Меркатто-Веккьо', pos='PROPN', feats=<Inan,Nom,Masc,Sing>, lemma='меркатто-веккьо'), DocToken(start=16, stop=17, text='-', pos='PUNCT', lemma='-'), DocToken(start=18, stop=26, text='торговый', pos='ADJ', feats=<Nom,Pos,Masc,Sing>, lemma='торговый'), DocToken(start=27, stop=32, text='центр', pos='NOUN', feats=<Inan,Nom,Masc,Sing>, lemma='центр'), DocToken(start=33, stop=42, text='Флоренции', pos='PROPN', feats=<Inan,Gen,Fem,Sing>, lemma='флоренция')]


## Syntax
Для каждого предложения запуускает анализатор синтаксиса. Зависит от глубины сегментации. Определяет `id`, `head_id`, `rel` свойства для `doc.tokens`. Использует **Slovnet syntax model** внутри себя.

In [19]:
doc.parse_syntax(syntax_parser)
print(doc.tokens[:5])

[DocToken(stop=15, text='Меркатто-Веккьо', id='1_1', head_id='1_4', rel='nsubj', pos='PROPN', feats=<Inan,Nom,Masc,Sing>, lemma='меркатто-веккьо'), DocToken(start=16, stop=17, text='-', id='1_2', head_id='1_4', rel='punct', pos='PUNCT', lemma='-'), DocToken(start=18, stop=26, text='торговый', id='1_3', head_id='1_4', rel='amod', pos='ADJ', feats=<Nom,Pos,Masc,Sing>, lemma='торговый'), DocToken(start=27, stop=32, text='центр', id='1_4', head_id='1_1', rel='appos', pos='NOUN', feats=<Inan,Nom,Masc,Sing>, lemma='центр'), DocToken(start=33, stop=42, text='Флоренции', id='1_5', head_id='1_4', rel='nmod', pos='PROPN', feats=<Inan,Gen,Fem,Sing>, lemma='флоренция')]


In [23]:
doc.sents[1].syntax.print()

      ┌──────────────► Антонио        nsubj
      │                Пуччи          
      │             ┌► ,              punct
    ┌►│             └─ поэт           conj
    │ │             ┌► ,              punct
    │ │ ┌────►┌─┌───└─ живший         acl
    │ │ │     │ │ ┌──► в              case
    │ │ │     │ │ │ ┌► XIV-м          amod
    │ │ │     │ └►└─└─ веке           obl
    │ │ │     └──────► ,              punct
    │ │ │           ┌► так            advmod
┌───│ └─│ ┌─┌─┌─┌─┌─└─ описал         
│   │   │ │ │ │ │ └──► Меркато-Веккьо obj
│   │   │ │ │ │ └────► :              punct
│   │   │ │ │ └──────► "              punct
│ ┌─└─┌─└─│ └────►┌─── Врачи          parataxis
│ │   │ │ │       │ ┌► ,              punct
│ │   │ │ │       └►└─ готовые        acl
│ │   │ │ └──────►┌─── исцелить       xcomp
│ │   │ │         │ ┌► любую          det
│ │   │ │         └►└─ хворь          obj
│ │   │ │           ┌► ,              punct
│ │   │ └──────►┌─┌─└─ торговцы       conj
│ │   │  

## NER
Извлекает стандартные именованные сущности: имена, локации, организации. Зависит от глубины сегментации. Определяет `spans` свойства для `doc`. Использует внутри **Slovnet NER mdeol**. 

In [24]:
doc.tag_ner(ner_tagger)

In [25]:
print(doc.spans[:5])

[DocSpan(stop=15, type='LOC', text='Меркатто-Веккьо', tokens=[...]), DocSpan(start=33, stop=42, type='LOC', text='Флоренции', tokens=[...]), DocSpan(start=44, stop=57, type='PER', text='Антонио Пуччи', tokens=[...]), DocSpan(start=97, stop=111, type='PER', text='Меркато-Веккьо', tokens=[...]), DocSpan(start=216, stop=231, type='LOC', text='Римской Империи', tokens=[...])]


In [26]:
doc.ner.print()

Меркатто-Веккьо - торговый центр Флоренции. Антонио Пуччи, поэт, 
LOC────────────                  LOC──────  PER──────────        
живший в XIV-м веке, так описал Меркато-Веккьо: "Врачи, готовые 
                                PER───────────                  
исцелить любую хворь, торговцы тканями, продавцы свиных колбас и 
аптекари". Во времена Римской Империи на месте Меркато был форум, и 
                      LOC────────────          PER────              
только в документах 1030 года он впервые назван рынком. Самым большим 
спросом здесь пользовались пшеница и шерсть. Однако если покупателям 
надоедало бродить по лавкам, они могли и развлечься. Иль-Панормита, 
                                                     PER──────────  
поэт XV века, написал об этом так: "Там, в глубине, стоит веселый 
бордель, который ты узнаешь по запаху".


## Named entity normalization
Для каждой ИС применима процедура нормализации. Зависит от ИС, морфологии и синаксичкской глубины. Определяет `normal` свойство для `doc.spans`.

Оно не просто лемматизирует каждый токен внутри `span`, чтобы "Организации украинских националистов" стала "Организация украинские националисты". Natasha использует синтаксическую взаимосвязь для корректного вывода "Организация украинских националистов".

In [27]:
for span in doc.spans:
    span.normalize(morph_vocab)
print(doc.spans[:5])
{_.text: _.normal for _ in doc.spans if _.text != _.normal}

[DocSpan(stop=15, type='LOC', text='Меркатто-Веккьо', tokens=[...], normal='Меркатто-Веккьо'), DocSpan(start=33, stop=42, type='LOC', text='Флоренции', tokens=[...], normal='Флоренция'), DocSpan(start=44, stop=57, type='PER', text='Антонио Пуччи', tokens=[...], normal='Антонио Пуччи'), DocSpan(start=97, stop=111, type='PER', text='Меркато-Веккьо', tokens=[...], normal='Меркато-Веккьо'), DocSpan(start=216, stop=231, type='LOC', text='Римской Империи', tokens=[...], normal='Римская Империя')]


{'Флоренции': 'Флоренция', 'Римской Империи': 'Римская Империя'}

## Named entity pasrsing
Парсинг ИС преобразуется в имя, фамилию и отчество. Зависит от шага **NER**. Определяет `fact` свойство для `doc.spans`. Использует **Yargy-parser** внутри себя.

In [28]:
for span in doc.spans:
    if span.type == PER:
        span.extract_fact(names_extractor)

print(doc.spans[:5])
{_.normal: _.fact.as_dict for _ in doc.spans if _.type == PER}

[DocSpan(stop=15, type='LOC', text='Меркатто-Веккьо', tokens=[...], normal='Меркатто-Веккьо'), DocSpan(start=33, stop=42, type='LOC', text='Флоренции', tokens=[...], normal='Флоренция'), DocSpan(start=44, stop=57, type='PER', text='Антонио Пуччи', tokens=[...], normal='Антонио Пуччи', fact=DocFact(slots=[...])), DocSpan(start=97, stop=111, type='PER', text='Меркато-Веккьо', tokens=[...], normal='Меркато-Веккьо', fact=DocFact(slots=[...])), DocSpan(start=216, stop=231, type='LOC', text='Римской Империи', tokens=[...], normal='Римская Империя')]


{'Антонио Пуччи': {'first': 'Антонио', 'last': 'Пуччи'},
 'Меркато-Веккьо': {'first': 'Меркато'},
 'Меркато': {'first': 'Меркато'},
 'Иль-Панормита': {'last': 'Иль'}}

## Documentation
- [Examples with description + reference](https://nbviewer.org/github/natasha/natasha/blob/master/docs.ipynb)
- [Natasha section in longread on Natasha project](https://habr.com/ru/post/516098/)
- [Natasha section of Datafest 2020 talk](https://www.youtube.com/watch?v=-7XT_U6hVvk&t=951s)